<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/descuentos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1067]:
!pip install xlsxwriter

In [1068]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import column_index_from_string, get_column_letter

In [1069]:
fecha_hoy = datetime.now().weekday()
if fecha_hoy == 0:
  fecha = str(datetime.strftime(datetime.now() - timedelta(2), "%d-%m-%Y"))
else:
  fecha = fecha = str(datetime.strftime(datetime.now() - timedelta(1), "%d-%m-%Y"))

archivo = "Reporte de descuentos-2" + fecha + ".xlsx"

In [1070]:
"""# Archivos necesarios"""

renglonaje = pd.read_excel("Renglonaje.xlsx", sheet_name=0)
ventas = pd.read_excel('Ventas.xlsx', sheet_name=0, header = 5)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [1071]:
descuentos = pd.read_excel('Descuentos.xlsx', sheet_name=0, header = 9)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [1072]:
try:
  ofertas = pd.read_excel('Ofertas.xlsx', sheet_name=0, header = 1)
  promos = ofertas[['Ean', 'Julio']]
  promos = promos.rename(columns={'Ean':'EAN', 'Julio':'Distribuidor'})
except:
  data = {'EAN':[-1.1], 'Distribuidor':[0]}
  promos = pd.DataFrame(data)

In [1073]:
pagos = descuentos.iloc[:, 5:].copy()

In [1074]:
pagos['Folio'] = pagos['Folio'].fillna(0)

In [1075]:
pagos.head()

,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Caja,Unnamed: 14,...,Unnamed: 23,Unnamed: 24,Descuento $\n,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Total
0,3551.0,NaN,BERTOLDO ANTONIO URNOSTEGUI VERONA,NaN,NaN,NaN,NaN,NaN,Caja 1,NaN,...,NaN,NaN,$ 19.50,NaT,NaN,NaN,NaN,NaN,NaN,$ 290.50
1,0.0,Formas de Pago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,Efectivo,NaN,Cheque,NaN,NaN,Crédito,NaN,NaN,NaN,...,Vales,NaN,NaN,NaT,NaN,NaN,Tarjeta,NaN,NaN,NaN
3,0.0,$ 0.00,NaN,$ 0.00,NaN,NaN,$ 0.00,NaN,NaN,NaN,...,$ 0.00,NaN,NaN,NaT,NaN,NaN,$ 290.50,NaN,NaN,NaN
4,0.0,Anticipo,NaN,SICAR Pagos,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [1076]:
folio = pagos['Folio'][0]
for i in pagos.index:
  if pagos['Folio'][i] != 0:
    folio = pagos['Folio'][i]
  else:
    pagos.loc[i, 'Folio'] = folio

In [1077]:
efectivo = pagos.iloc[3::6, :2]
efectivo = efectivo.dropna()
efectivo = efectivo.reset_index(drop=True)
efectivo = efectivo.rename(columns={'Unnamed: 6': 'Efectivo'})

In [1078]:
cheque = pagos.iloc[3::6, :4:3]
cheque = cheque.dropna()
cheque = cheque.reset_index(drop=True)
cheque = cheque.rename(columns={'Unnamed: 8': 'Cheque'})

In [1079]:
credito = pagos.iloc[3::6, :7:6]
credito = credito.dropna()
credito = credito.reset_index(drop=True)
credito = credito.rename(columns={'Unnamed: 11': 'Crédito'})

In [1080]:
transf = pagos.iloc[3::6, :11:10]
transf = transf.dropna()
transf = transf.reset_index(drop=True)
transf = transf.rename(columns={'Unnamed: 15': 'Transferencia'})

In [1081]:
vales = pagos.iloc[3::6, :19:18]
vales = vales.dropna()
vales = vales.reset_index(drop=True)
vales = vales.rename(columns={'Unnamed: 23': 'Vales'})

In [1082]:
tarjeta = pagos.iloc[3::6, :25:24]
tarjeta = tarjeta.dropna()
tarjeta = tarjeta.reset_index(drop=True)
tarjeta = tarjeta.rename(columns={'Unnamed: 29': 'Tarjeta'})

In [1083]:
anticipo = pagos.iloc[5::6, :2]
anticipo = anticipo.dropna()
anticipo = anticipo.reset_index(drop=True)
anticipo = anticipo.rename(columns={'Unnamed: 6': 'Anticipo'})

In [1084]:
sicar = pagos.iloc[5::6, :4:3]
sicar = sicar.dropna()
sicar = sicar.reset_index(drop=True)
sicar = sicar.rename(columns={'Unnamed: 8': 'SICAR'})

In [1085]:
efectivo['Efectivo'] = efectivo['Efectivo'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
cheque['Cheque'] = cheque['Cheque'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
credito['Crédito'] = credito['Crédito'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
transf['Transferencia'] = transf['Transferencia'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
vales['Vales'] = vales['Vales'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
tarjeta['Tarjeta'] = tarjeta['Tarjeta'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
anticipo['Anticipo'] = anticipo['Anticipo'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
sicar['SICAR'] = sicar['SICAR'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

In [1086]:
ventas = ventas.merge(efectivo, on='Folio', how='left')
ventas = ventas.merge(cheque, on='Folio', how='left')
ventas = ventas.merge(credito, on='Folio', how='left')
ventas = ventas.merge(transf, on='Folio', how='left')
ventas = ventas.merge(vales, on='Folio', how='left')
ventas = ventas.merge(tarjeta, on='Folio', how='left')
ventas = ventas.merge(anticipo, on='Folio', how='left')
ventas = ventas.merge(sicar, on='Folio', how='left')

In [1087]:
ventas.head()

,Documento,Unnamed: 1,Unnamed: 2,Fecha,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Total,Unnamed: 31,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR
0,Ticket,NaN,NaN,2025-07-26 08:31:55,NaN,3551.0,NaN,BERTOLDO ANTONIO URNOSTEGUI VERONA,NaN,NaN,...,$ 290.50,NaN,0.0,0.0,0.0,0.0,0.0,290.5,0.0,0.0
1,PZA,1.0,NaN,NaT,[7501206699867] 19881 Escoba plástica p/jardín...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PZA,1.0,NaN,NaT,[7506240618973] 10567 Casco de seguridad ala a...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ticket,NaN,NaN,2025-07-26 08:51:34,NaN,3552.0,NaN,Público en General,NaN,NaN,...,"$ 2,250.00",NaN,0.0,0.0,0.0,0.0,0.0,2250.0,0.0,0.0
4,PZA,1.0,NaN,NaT,[7506240643807] 12498 Desbrozadora a gasolina ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1088]:
df = ventas

In [1089]:
df['Documento'] = df['Documento'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Fecha'] = df['Fecha'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Cliente'] = df['Cliente'].fillna(0)
df['Caja'] = df['Caja'].fillna(0)
df['Usuario'] = df['Usuario'].fillna(0)
df['Est'] = df['Est'].fillna(0)

In [1090]:
lista = ['Unnamed: 2', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 18', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 31']
df = df.drop(lista, axis=1)

In [1091]:
df.insert(1, 'Unidad', 0)

In [1092]:
df['Unidad'] = df['Documento']
df['Unidad'] = df['Unidad'].replace('Ticket', '--------')
df['Unidad'] = df['Unidad'].replace('Factura CFDI', '--------')

In [1093]:
df = df.rename(columns={'Unnamed: 1':'Cantidad', 'Unnamed: 4': 'Producto', 'Unnamed: 19': 'Precio Unitario'})

In [1094]:
df['Producto'] = df['Producto'].fillna(0)

In [1095]:
desc = descuentos[['Folio', 'Desc %', 'Descuento $\n']]
desc = desc.dropna()
desc = desc.reset_index(drop=True)
df = df.merge(desc, on='Folio', how='left')

In [1096]:
documento = df['Documento'][0]
for i in df.index:
  if df['Documento'][i] == 'Ticket' or df['Documento'][i] == 'Factura CFDI':
    documento = df['Documento'][i]
  else:
    df.loc[i, 'Documento'] = documento

fecha = df['Fecha'][0]
for i in df.index:
  if df['Fecha'][i] != 0:
    fecha = df['Fecha'][i]
  else:
    df.loc[i, 'Fecha'] = fecha

cliente = df['Cliente'][0]
for i in df.index:
  if df['Cliente'][i] != 0:
    cliente = df['Cliente'][i]
  else:
    df.loc[i, 'Cliente'] = cliente

caja = df['Caja'][0]
for i in df.index:
  if df['Caja'][i] != 0:
    caja = df['Caja'][i]
  else:
    df.loc[i, 'Caja'] = caja

usuario = df['Usuario'][0]
for i in df.index:
  if df['Usuario'][i] != 0:
    usuario = df['Usuario'][i]
  else:
    df.loc[i, 'Usuario'] = usuario

est = df['Est'][0]
for i in df.index:
  if df['Est'][i] != 0:
    est = df['Est'][i]
  else:
    df.loc[i, 'Est'] = est

folio = df['Folio'][0]
for i in df.index:
  if df['Folio'][i] != 0:
    folio = df['Folio'][i]
  else:
    df.loc[i, 'Folio'] = folio

In [1097]:
df.insert(4, 'EAN', 0)
df.insert(5, 'Clave', 0)
df.insert(6, 'Descripción', 0)

In [1098]:
df = df.drop(df.tail(17).index)

In [1099]:
for i in df.index:
  producto = df['Producto'][i]
  if producto != 0:
    a = producto.split(']')
    ean = a[0][1:]
    b = a[1]
    for j in range(len(b)):
      if b[j].isalpha():
        clave = b[1:j]
        descripcion = b[j:]
        break
    df.loc[i, 'EAN'] = ean
    df.loc[i, 'Clave'] = clave
    df.loc[i, 'Descripción'] = descripcion

df = df.drop('Producto', axis=1)

/tmp/ipython-input-1099-3600708791.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7501206699867' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'EAN'] = ean
/tmp/ipython-input-1099-3600708791.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '19881 ' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Clave'] = clave
/tmp/ipython-input-1099-3600708791.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Escoba plástica p/jardín, flex. 26 ' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Descripción'] = descripcion


In [1100]:
df.insert(1, "Día", "")
df.insert(2, "Hora", "")

df['Fecha'] = df['Fecha'].astype(str)

for i in df.index:
  fecha = df['Fecha'][i].split(' ')
  df.loc[i, 'Día'] = fecha[0]
  df.loc[i, 'Hora'] = fecha[1]

df = df.drop('Fecha', axis=1)
df = df.rename(columns={'Día':'Fecha'})

In [1101]:
df['Precio Unitario'] = df['Precio Unitario'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df = df.rename(columns={'Descuento $\n':'Descuento $'})
df['Descuento $'] = df['Descuento $'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos = df['Desc %'].str.strip().str.replace('%', '').astype(float)

In [1102]:
df = df.rename(columns={'Total   ': 'Total'})

In [1103]:
df['Total'] = df['Total'].fillna(0)
df['Total'] = df['Total'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

In [1104]:
df['Total'] = df['Total'].fillna(0)

In [1105]:
df['Precio Unitario'] = df['Precio Unitario'].fillna(0)
df['Cantidad'] = df['Cantidad'].fillna(1)
df['Total'] = df['Total'] + df['Precio Unitario'] * df['Cantidad']

In [1106]:
"""Completamos la columna de "Total"
"""

df['Efectivo'] = df['Efectivo'].fillna(-1)
df['Cheque'] = df['Cheque'].fillna(-1)
df['Crédito'] = df['Crédito'].fillna(-1)
df['Transferencia'] = df['Transferencia'].fillna(-1)
df['Vales'] = df['Vales'].fillna(-1)
df['Tarjeta'] = df['Tarjeta'].fillna(-1)
df['Anticipo'] = df['Anticipo'].fillna(-1)
df['SICAR'] = df['SICAR'].fillna(-1)

In [1107]:
efectivo = df['Efectivo'][0]
for i in df.index:
  if df['Efectivo'][i] != -1:
    if df['Efectivo'][i] == 0:
      efectivo = np.nan
    else:
      efectivo = 1
      df.loc[i, 'Efectivo'] = efectivo
  else:
    df.loc[i, 'Efectivo'] = efectivo

In [1108]:
"""Para evitar el ruido visual, las tendremos columnas vacías en los datos financieros que tengan 0 en su valor total"""


df['Efectivo'] = df['Efectivo'] * df['Total']

cheque = df['Cheque'][0]
for i in df.index:
  if df['Cheque'][i] != -1:
    if df['Cheque'][i] == 0:
      cheque = np.nan
    else:
      cheque = 1
      df.loc[i, 'Cheque'] = cheque
  else:
    df.loc[i, 'Cheque'] = cheque

df['Cheque'] = df['Cheque'] * df['Total']

credito = df['Crédito'][0]
for i in df.index:
  if df['Crédito'][i] != -1:
    if df['Crédito'][i] == 0:
      credito = np.nan
    else:
      credito = 1
      df.loc[i, 'Crédito'] = credito
  else:
    df.loc[i, 'Crédito'] = credito

df['Crédito'] = df['Crédito'] * df['Total']

transferencia = df['Transferencia'][0]
for i in df.index:
  if df['Transferencia'][i] != -1:
    if df['Transferencia'][i] == 0:
      transferencia = np.nan
    else:
      transferencia = 1
      df.loc[i, 'Transferencia'] = transferencia
  else:
    df.loc[i, 'Transferencia'] = transferencia

df['Transferencia'] = df['Transferencia'] * df['Total']

vales = df['Vales'][0]
for i in df.index:
  if df['Vales'][i] != -1:
    if df['Vales'][i] == 0:
      vales = np.nan
    else:
      vales = 1
      df.loc[i, 'Vales'] = vales
  else:
    df.loc[i, 'Vales'] = vales

df['Vales'] = df['Vales'] * df['Total']

tarjeta = df['Tarjeta'][0]
for i in df.index:
  if df['Tarjeta'][i] != -1:
    if df['Tarjeta'][i] == 0:
      tarjeta = np.nan
    else:
      tarjeta = 1
      df.loc[i, 'Tarjeta'] = tarjeta
  else:
    df.loc[i, 'Tarjeta'] = tarjeta

df['Tarjeta'] = df['Tarjeta'] * df['Total']

anticipo = df['Anticipo'][0]
for i in df.index:
  if df['Anticipo'][i] != -1:
    if df['Anticipo'][i] == 0:
      anticipo = np.nan
    else:
      anticipo = 1
      df.loc[i, 'Anticipo'] = anticipo
  else:
    df.loc[i, 'Anticipo'] = anticipo

df['Anticipo'] = df['Anticipo'] * df['Total']

sicar_pagos = df['SICAR'][0]
for i in df.index:
  if df['SICAR'][i] != -1:
    if df['SICAR'][i] == 0:
      sicar_pagos = np.nan
    else:
      sicar_pagos = 1
      df.loc[i, 'SICAR'] = sicar_pagos
  else:
    df.loc[i, 'SICAR'] = sicar_pagos

df['SICAR'] = df['SICAR'] * df['Total']

"""Completamos la columna de descuentos"""

df['Desc %'] = df['Desc %'].fillna(0)
descuento = df['Desc %'][0]
for i in df.index:
  if df['Desc %'][i] != 0:
    descuento = df['Desc %'][i]
  else:
    df.loc[i, 'Desc %'] = descuento

In [1109]:
df['Unidad'] = df['Unidad'].replace(' ', '----------')
df['Cantidad'] = df['Cantidad'].fillna('----------')
df['Precio Unitario'] = df['Precio Unitario'].replace(0, '----------')
df['Clave'] = df['Clave'].replace(0, '----------')
df['EAN'] = df['EAN'].replace(0, '----------')
df['Descripción'] = df['Descripción'].replace(0, '----------')
df['Folio F.'] = df['Folio F.'].fillna('')

In [1110]:
descuentos = df[['Folio', 'Cliente','Clave', 'EAN', 'Desc %', 'Precio Unitario', 'Cantidad', 'Total']]

In [1111]:
"""Filtramos los productos"""

descuentos = descuentos.loc[descuentos['Clave'] != '----------']
descuentos['Clave'] = descuentos['Clave'].replace('', 0)
descuentos['EAN'] = pd.to_numeric(descuentos['EAN'], errors='coerce')
descuentos = descuentos.reset_index(drop=True)

"""Extraemos los datos de renglonaje, de la hoja del catálogo de Truper"""

costos = renglonaje[['ean', 'precio mayoreo con IVA', 'precio mínimo de venta', 'precio público con IVA']]
costos = costos.rename(columns={'ean':'EAN'})

costos['precio mayoreo con IVA'] = costos['precio mayoreo con IVA'].replace('*', 0)
costos['EAN'] = costos['EAN'].fillna(0)
costos['EAN'] = costos['EAN'].replace('', 0)
costos['EAN'] = pd.to_numeric(costos['EAN'], errors='coerce')

descuentos = descuentos.merge(costos, on='EAN', how='left')
descuentos = descuentos.merge(promos, on='EAN', how='left')

descuentos['Diferencia mayoreo'] = descuentos['Precio Unitario'] - descuentos['precio mayoreo con IVA']
descuentos['Diferencia minimo'] = descuentos['Precio Unitario'] - descuentos['precio mínimo de venta']
descuentos['Diferencia publico'] = descuentos['Precio Unitario'] - descuentos['precio público con IVA']
descuentos['Diferencia dist.'] = descuentos['Precio Unitario'] - descuentos['Distribuidor']

"""Creación del excel"""

writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

df.to_excel(writer, sheet_name='SICAR', index=False)
descuentos.to_excel(writer, sheet_name='Reporte', index=False)

writer.close()

"""# Dando formato al Excel

## Hoja de SICAR
"""

wb = load_workbook(archivo)
ws = wb['SICAR']
last_col = column_index_from_string('P')
col_z = column_index_from_string('Z')
col_dif_mayoreo = column_index_from_string('M')
col_dif_min = column_index_from_string('N')
col_dif_pub = column_index_from_string('O')
col_dif_dist = column_index_from_string('P')

"""### Fuentes
- font_t = Fuente para la fila de metadatos
- font = Fuente para el resto de documentos
"""

font_t = Font(name='Comfortaa', size=12, bold=True, italic=False, color='ffffff')
font = Font(name='Comfortaa', size=11, bold=False, italic=False, color='000000')

"""### Colores de fondo
- fill_b = negro
- fill_r = rojo
- fill_g = verde
- fill_y = amarillo
- fill_p = morado
- fill_bl = azul
"""

fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")
fill_r = PatternFill(fill_type="solid", start_color="ffc7ce", end_color="ffc7ce")
fill_g = PatternFill(fill_type="solid", start_color="b6d7a8", end_color="b6d7a8")
fill_y = PatternFill(fill_type="solid", start_color="ffe599", end_color="ffe599")
fill_p = PatternFill(fill_type="solid", start_color="b4a7d6", end_color="b4a7d6")
fill_bl = PatternFill(fill_type="solid", start_color="a4c2f4", end_color="a4c2f4")

for col in range(1,col_z + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

ws.column_dimensions['F'].width = 40

for col in range(1, 27):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='center')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, 27):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='center')

for row in range(2, 1000):
  if ws.cell(row=row, column=16).value == None:
    break
  ws.cell(row=row, column=16).fill = fill_g

num_form = '$ #,##0.00'
for row in range(2, 1000):
  for col in range(15, 28):
    ws.cell(row=row, column=col).number_format = num_form

wb.save(archivo)

"""## Hoja de Reporte"""

ws = wb['Reporte']

for col in range(1, last_col + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

for col in range(1, last_col + 1):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='right')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, last_col + 1):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='right')

"""### Explicación del ciclo for:

Por cada fila desde la 2 hasta la 1000:

---

- SI no hay número de folio, ROMPE CICLO
- SI el valor de diferencia entre el precio y el precio mínimo de venta es nulo, pinta de morado; SI NO -> SI la diferencia con el mayoreo y la diferencia con el mínimo de venta es diferente de 0 -> A

---

A

- SI el valor de descuento es 0.00% -> B; SI NO -> Pinta de amarillo toda la fila

---

B

- SI la diferencia con el precio distribuidor es diferente de 0, pinta de rojo; SI NO, pinta de azul

---
Los colores significan lo siguiente:
---
- El color rojo representa que se dió a un precio menor al mínimo SIN descuento o promoción.

- El color amarillo significa que se dió un precio menor al mínimo CON descuento.

- El color morado representa un error o una fila atípica.

- El azul representa que se vendió un producto con oferta/promoción
"""

for row in range(2, 1000):
  if(ws.cell(row=row, column=1).value == None):
    break

  dif_may = ws.cell(row=row, column=col_dif_mayoreo).value
  dif_min = ws.cell(row=row, column=col_dif_min).value
  dif_pub = ws.cell(row=row, column = col_dif_pub).value
  dif_dist = ws.cell(row=row, column=col_dif_dist).value
  descuento = ws.cell(row=row, column=5).value

  if(dif_min == None):
    for col in range(1, last_col + 1):
      ws.cell(row=row, column=col).fill = fill_p
  elif(dif_may != 0 and dif_min != 0 and dif_pub != 0):
    if(descuento == '0.00%'):
      if(dif_dist != 0):
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_r
      else:
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_bl
    else:
      for col in range(1, last_col + 1):
        ws.cell(row=row, column=col).fill = fill_y

"""Le da formato de número a las filas de datos financieros"""

num_form = '$ #,##0.00'
for row in range(2, 1000):
  ws.cell(row=row, column=6).number_format = num_form
  for col in range(8, last_col + 1):
    ws.cell(row=row, column=col).number_format = num_form

wb.save(archivo)

/tmp/ipython-input-1111-3128928496.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  costos['precio mayoreo con IVA'] = costos['precio mayoreo con IVA'].replace('*', 0)
